In [ ]:
import sys
sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/')

import scanpy as sc
import time 
import os.path as op
import os
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import scipy.sparse as scs

import torch
import yaml


from netmap.src.utils.misc import write_config


from netmap.src.utils.data_utils import *
from netmap.src.utils.tf_utils import *
from netmap.src.utils.netmap_config import NetmapConfig

from netmap.src.model.train_model import create_model_zoo
from netmap.src.grn.inferrence import inferrence
from src.data_simulation.data_simulation_config import DataSimulationConfig
from netmap.src.masking.internal import *
from netmap.src.masking.external import *

def read_config(file):
    with open(file, "r") as f:
        config = yaml.safe_load(f)
    return config





In [ ]:



#config = NetmapConfig.read_yaml("/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/netmap/config/perturb_seq/")
dada = "/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/data_simulation/config_easy/net_172_54892_net_131_54992_net_158_55084.config.yaml"
dataset_config = read_config("/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/data_simulation/config_easy/net_172_54892_net_131_54992_net_158_55084.config.yaml")

nets = [pd.read_csv(op.join("/data_nfs/og86asub/netmap/netmap-evaluation/data/clustered_network/", filename), sep='\t') for filename in dataset_config['edgelist']]
common = [pd.read_csv(op.join("/data_nfs/og86asub/netmap/netmap-evaluation/data/clustered_network/", filename), sep='\t') for filename in dataset_config['common_edges']]

    
config = NetmapConfig.read_yaml('/data_nfs/og86asub/netmap/netmap-evaluation/results/netmap/config_22/config_easy/net_172_54892_net_131_54992_net_158_55084/config.yaml')
dataset_config = DataSimulationConfig.read_yaml(dada)



start_total = time.monotonic()

## Load config and setup outputs
os.makedirs(config.output_directory, exist_ok=True)
sc.settings.figdir = config.output_directory
config.write_yaml(yaml_file=op.join(config.output_directory, 'config.yaml'))

## load data
adata = sc.read_h5ad(config.input_data)


## Get the data matrix from the CustumAnndata obeject
gene_names = np.array(adata.var.index)
model_start = time.monotonic()

if config.layer == 'counts':
    data_tensor = adata.layers['counts']
else:
    data_tensor = adata.X

if scs.issparse(data_tensor):
    data_tensor = torch.tensor(data_tensor.todense(), dtype=torch.float32)
else:
    data_tensor = torch.tensor(data_tensor, dtype=torch.float32)


{'input_data': '/data_nfs/og86asub/netmap/netmap-evaluation/data/simulated_data/config_easy/net_172_54892_net_131_54992_net_158_55084/data.h5ad', 'layer': 'X', 'output_directory': '/data_nfs/og86asub/netmap/netmap-evaluation/results/netmap/config_22/config_easy/net_172_54892_net_131_54992_net_158_55084', 'transcription_factors': '/data_nfs/datasets/SCENIC_DB/tf_lists/allTFs_hg38.txt', 'tf_only': False, 'penalize_error': True, 'adata_filename': 'grn_lrp.h5ad', 'grn': 'grn_lrp.tsv', 'masking_percentage': 0.1, 'masking_value': 0, 'print_every': 100, 'optimizer': 'Adam', 'learning_rate': 0.005, 'epochs': 150, 'overwrite': True, 'n_models': 10, 'n_top_edges': 100, 'test_size': 0.3, 'edge_count': 10000, 'model': 'negbin', 'loss_fn': 'LogCoshLoss', 'xai_method': 'GuidedBackprop', 'top_perc': False, 'percentile': 50, 'penalty': 'None', 'raw_attribution': False, 'aggregation_strategy': 'median', 'use_differential': False}


In [ ]:
model_zoo = create_model_zoo(data_tensor,  n_models=config.n_models, n_epochs=config.epochs, model_type=config.model)

grn_adata = inferrence(model_zoo, data_tensor.cuda(), gene_names,  config, use_raw_attribution=False)
grn_adata_raw = inferrence(model_zoo, data_tensor.cuda(), gene_names,  config, use_raw_attribution=True)
grn_adata.layers['raw_attribution'] = grn_adata_raw.X
grn_adata.layers['raw_attribution_quantile_count'] = grn_adata_raw.layers['quantile_count']

grn_adata = add_neighbourhood_expression_mask(adata,grn_adata)

Epoch 1/150, Train Loss: 51.5219, Validation Loss: 33.8597
Epoch 1/150, Train Loss: 51.5219
Epoch 2/150, Train Loss: 28.5506
Epoch 3/150, Train Loss: 17.2941
Epoch 4/150, Train Loss: 9.8294
Epoch 5/150, Train Loss: 7.6708
Epoch 6/150, Train Loss: 7.0387
Epoch 7/150, Train Loss: 6.6458
Epoch 8/150, Train Loss: 6.4221
Epoch 9/150, Train Loss: 6.2168
Epoch 10/150, Train Loss: 6.0646
Epoch 11/150, Train Loss: 5.9334, Validation Loss: 5.7267
Epoch 11/150, Train Loss: 5.9334
Epoch 12/150, Train Loss: 5.7994
Epoch 13/150, Train Loss: 5.7075
Epoch 14/150, Train Loss: 5.5892
Epoch 15/150, Train Loss: 5.5081
Epoch 16/150, Train Loss: 5.4111
Epoch 17/150, Train Loss: 5.3264
Epoch 18/150, Train Loss: 5.2373
Epoch 19/150, Train Loss: 5.1650
Epoch 20/150, Train Loss: 5.1050
Epoch 21/150, Train Loss: 5.0300, Validation Loss: 4.8974
Epoch 21/150, Train Loss: 5.0300
Epoch 22/150, Train Loss: 4.9660
Epoch 23/150, Train Loss: 4.9106
Epoch 24/150, Train Loss: 4.8453
Epoch 25/150, Train Loss: 4.8158
Epoch 

100%|██████████| 461/461 [02:06<00:00,  3.65it/s]


Creating anndata
Setting vars


In [12]:

pariwise_consistency_1 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 1.0, :].copy(), gene_names)
pariwise_consistency_2 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 2.0, :].copy(), gene_names)

In [13]:
pairwise_consistency_1 = pd.DataFrame(pariwise_consistency_1.items())
pairwise_consistency_2 = pd.DataFrame(pariwise_consistency_2.items())

In [14]:
pairwise_consistency_1.columns = ['edge', 'spearman']
pairwise_consistency_2.columns = ['edge', 'spearman']



In [15]:
pairwise_consistency_1

,edge,spearman
0,AATF_ABCA1,0.282
1,ABCA1_AATF,0.282
2,AATF_ABCA12,-0.404
3,ABCA12_AATF,-0.404
4,AATF_ABCB1,-0.591
...,...,...
212055,ZNF692_ZNF513,0.787
212056,ZNF513_ZNF746,-0.230
212057,ZNF746_ZNF513,-0.230
212058,ZNF692_ZNF746,-0.221


In [16]:
def get_top_edges_per_cell(grn_adata, top_edges):
    
    idex = grn_adata.shape[1]-top_edges
    b = np.argpartition(grn_adata.X, idex, axis=1)    # top 3 values from each row
    top_idx = b[:,idex:]


    scgenerai_var_index_np = np.array(grn_adata.var)
    top_edges_per_cell = pd.DataFrame(np.concat(scgenerai_var_index_np[top_idx]))
    top_edges_per_cell.columns = ['source', 'target', 'n_cells']
    top_edges_per_cell['cell_barcode'] = np.repeat(grn_adata.obs.index, repeats=top_edges)
    return top_edges_per_cell



